# SB1 Big Data Downloads
An analysis into the corridor analysis data downloads from Streetlight and Replica

In [1]:
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

import altair as alt
from calitp_data_analysis import calitp_color_palette as cp

from IPython.display import Markdown, HTML, display_html, display
from IPython.core.display import display

import numpy as np

/tmp/ipykernel_305/4150955979.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [76]:
# ! pip install nltk

In [77]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import re

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)

In [3]:
GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/corridor_study_data/"

### Read in Replica Data Spring 2023

In [4]:
replica_trips = "replica-bay_bridge-03_06_24-trips_dataset.csv"

In [5]:
replica_ppl = "replica-bay_bridge-03_06_24-people_dataset.csv"

In [6]:
r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))

/tmp/ipykernel_305/2643890524.py:1: DtypeWarning: Columns (2,4,5,9,10,11,18,25,26,29,30,35,36,40,43,48,50) have mixed types. Specify dtype option on import or set low_memory=False.
  r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))


In [7]:
display(HTML("<h4><strong> Sample of data</strong></h4>"))


In [8]:
r_trips.sample(5)

,origin_building_use,trip_duration_minutes,trip_taker_language,origin_trct_2020,trip_taker_household_id,trip_taker_sex,trip_taker_home_bgrp_2020,origin_st_2020,trip_taker_home_st_2020,trip_taker_race_ethnicity,transit_agency,transit_route,trip_taker_resident_type,vehicle_type,trip_taker_age,activity_id,destination_bgrp_2020,destination_building_use,trip_taker_wfh,destination_st_2020,destination_cty_2020,trip_taker_work_bgrp_2020,destination_trct_2020,trip_taker_household_income,origin_bgrp_2020,trip_taker_commute_mode,trip_taker_available_vehicles,primary_mode,previous_trip_purpose,trip_taker_building_type,transit_submode,trip_taker_work_st_2020,trip_taker_household_size,origin_cty_2020,destination_land_use,trip_taker_industry,trip_taker_tenure,vehicle_fuel_type,trip_taker_home_trct_2020,trip_taker_work_trct_2020,trip_taker_education,trip_start_time,trip_taker_individual_income,trip_taker_employment_status,trip_purpose,origin_land_use,trip_taker_work_cty_2020,trip_distance_miles,trip_taker_school_grade_attending,trip_taker_home_cty_2020,trip_taker_person_id,trip_end_time
192322,multi_family,42,indo_european,"4054.02 (Alameda, CA)",2985112111348362186,female,"1 (Tract 4054.02, Alameda, CA)",California,California,white_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,36.0,4961260832094318244,"1 (Tract 127, San Francisco, CA)",retail,in_person,California,San Francisco,"4 (Tract 4371.01, Alameda, CA)","127 (San Francisco, CA)",91622.0,"1 (Tract 4054.02, Alameda, CA)",public_transit,two,auto_passenger,home,several_units,NaN,California,2.0,Alameda,retail,naics31_33,renter,unknown_fuel_type,"4054.02 (Alameda, CA)","4371.01 (Alameda, CA)",advanced_degree,17:56:00,28434.0,employed,eat,multi_family,Alameda,17.6,not_attending_school,Alameda,11571861350915216215,18:38:59
556927,multi_family,50,spanish,"3240.03 (Contra Costa, CA)",12416463756595895517,female,"2 (Tract 3240.03, Contra Costa, CA)",California,California,hispanic_or_latino_origin,AC TRANSIT,Pleas Hill Transbay Early Bird,core,unknown_vehicle_type,29.0,3465375536630884598,"2 (Tract 105, San Francisco, CA)",retail,in_person,California,San Francisco,"2 (Tract 105, San Francisco, CA)","105 (San Francisco, CA)",228019.0,"2 (Tract 3240.03, Contra Costa, CA)",public_transit,one,public_transit,home,multiple_units,bus,California,2.0,Contra Costa,retail,naics451110,renter,unknown_fuel_type,"3240.03 (Contra Costa, CA)","105 (San Francisco, CA)",bachelors_degree,04:15:00,128461.0,employed,work,multi_family,San Francisco,21.6,not_attending_school,Contra Costa,15705699079925857553,05:05:44
476119,retail,54,english,"5046.02 (Santa Clara, CA)",3308794688161543955,male,"3 (Tract 119.01, San Francisco, CA)",California,California,black_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,51.0,13742818899929327986,"3 (Tract 119.01, San Francisco, CA)",multi_family,in_person,California,San Francisco,"1 (Tract 5046.02, Santa Clara, CA)","119.01 (San Francisco, CA)",269769.0,"1 (Tract 5046.02, Santa Clara, CA)",private_auto,two,private_auto,work,multiple_units,NaN,California,1.0,Santa Clara,multi_family,naics442110,renter,other_non_bev,"119.01 (San Francisco, CA)","5046.02 (Santa Clara, CA)",bachelors_degree,19:12:00,269769.0,employed,home,retail,Santa Clara,45.0,not_attending_school,San Francisco,8753623118396865231,20:06:30
604416,retail,17,english,"4011 (Alameda, CA)",5023938673641210752,male,"3 (Tract 254.02, San Francisco, CA)",California,California,other_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,35.0,17759737250616177446,"3 (Tract 254.02, San Francisco, CA)",single_family,remote,California,San Francisco,"3 (Tract 176.02, San Francisco, CA)","254.02 (San Francisco, CA)",23551.0,"1 (Tract 4011, Alameda, CA)",worked_from_home,one,private_auto,eat,single_family,NaN,California,3.0,Alameda,single_family,naics56,renter,other_non_bev,"254.02 (San Francisco, CA)","176.02 (San Francisco, CA)",high_school,20:08:00,10491.0,employed,home,retail,San Francisco,12.9,not_attending_sc

In [9]:
display(HTML("<h4><strong> Columns in Replica Trips Data</strong></h4>"))


In [10]:
r_trips.columns

Index(['origin_building_use', 'trip_duration_minutes', 'trip_taker_language',
       'origin_trct_2020', 'trip_taker_household_id', 'trip_taker_sex',
       'trip_taker_home_bgrp_2020', 'origin_st_2020',
       'trip_taker_home_st_2020', 'trip_taker_race_ethnicity',
       'transit_agency', 'transit_route', 'trip_taker_resident_type',
       'vehicle_type', 'trip_taker_age', 'activity_id',
       'destination_bgrp_2020', 'destination_building_use', 'trip_taker_wfh',
       'destination_st_2020', 'destination_cty_2020',
       'trip_taker_work_bgrp_2020', 'destination_trct_2020',
       'trip_taker_household_income', 'origin_bgrp_2020',
       'trip_taker_commute_mode', 'trip_taker_available_vehicles',
       'primary_mode', 'previous_trip_purpose', 'trip_taker_building_type',
       'transit_submode', 'trip_taker_work_st_2020',
       'trip_taker_household_size', 'origin_cty_2020', 'destination_land_use',
       'trip_taker_industry', 'trip_taker_tenure', 'vehicle_fuel_type',
       't

#### Data Exploration: Replica Trips
* Summarizing the data that we exported from Replica
* Existing visualizations in replica: 
   * Primary Mode
   * Trip Purpose
   * Starting hour 
   * Trip Duration/Distance
   * Origin Destination
   * Vehicle Fuel type
   * Transit Routes/Stops/Sub mode/Agency
   * Household Income
   * Race and Ethnicity
   * Private Auto Availability
   * Age
   * Employment/School Status
   
   

In [11]:
trips = r_trips>>group_by(_.primary_mode)>>summarize(avg_trip_time = _.trip_duration_minutes.mean(),
                                                     number_trips = _.activity_id.nunique())

In [12]:
display(HTML("<h4><strong>Average Trip Time</strong></h4>"))


In [13]:
chart = (
        alt.Chart(trips)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("avg_trip_time"),
            color=alt.Color("avg_trip_time", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS))
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [14]:
display(HTML("<h4><strong>Number of Trips</strong></h4>"))


In [15]:
chart = (
        alt.Chart(trips)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("number_trips"),
            color=alt.Color("number_trips", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,))
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [16]:
display(HTML("<h4><strong>Length of Trips dataframe</strong></h4>"))

In [17]:
len(r_trips)

674864

In [18]:
display(HTML("<h4><strong>Number of unique activity ids in data</strong></h4>"))

In [19]:
r_trips>>summarize(number_trips = _.activity_id.nunique())

,number_trips
0,674864


In [20]:
display(HTML("<h4><strong>Number of unique trip taker ids</strong></h4>")) 

In [21]:
r_trips>>summarize(number_trips = _.trip_taker_person_id.nunique())

,number_trips
0,309596


In [22]:
race_ethnicity_by_mode = r_trips>>group_by(_.primary_mode)>>count(_.trip_taker_race_ethnicity)

In [23]:
display(HTML("<h4><strong>Trip Taker Race and Ethnicity by Mode</strong></h4>")) 

In [24]:
chart = (
        alt.Chart(race_ethnicity_by_mode)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("n"),
            color=alt.Color("trip_taker_race_ethnicity", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
                                                                         domain=race_ethnicity_by_mode["trip_taker_race_ethnicity"].unique().tolist())
            ),
            tooltip=race_ethnicity_by_mode.columns.tolist())
            
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [25]:
display(HTML("<H4><strong>Transit Mode Splits</strong></H4>")) 


In [26]:
display(HTML("<strong>Looking at transit mode, need to alter the transit_agency col and transit_submode to get a better count.</strong>")) 


In [27]:
ptt_agency_count = (r_trips
 >>filter(_.primary_mode=="public_transit")
 >>group_by(_.primary_mode, _.transit_submode)
 >>count(_.transit_agency)
 >>arrange(-_.n))

In [28]:
ptt_agency_count

,primary_mode,transit_submode,transit_agency,n
0,public_transit,bus,AC TRANSIT,2788
12,public_transit,"bus, bus","San Francisco Municipal Transportation Agency, AC TRANSIT",632
8,public_transit,"bus, bus","AC TRANSIT, San Francisco Municipal Transportation Agency",516
6,public_transit,"bus, bus","AC TRANSIT, AC TRANSIT",263
75,public_transit,"bus, light_rail","AC TRANSIT, San Francisco Municipal Transportation Agency",253
...,...,...,...,...
128,public_transit,"rail, bus","Caltrain, SamTrans",1
130,public_transit,"rail, bus, bus","Amtrak, AC TRANSIT, AC TRANSIT",1
133,public_transit,"rail, light_rail, bus","Caltrain, San Francisco Municipal Transportation Agency, AC TRANSIT",1
136,public_transit,"rail, subway, bus, bus","Caltrain, Bay Area Rapid Transit, San Francisco Municipal Transportation Agency, AC TRANSIT",1


In [29]:
display(HTML("<h4><strong>Most common transit mode combinations</strong></h4>")) 


In [30]:
ptt_modes = (r_trips
             >>filter(_.primary_mode =="public_transit")
             >>count(_.transit_submode)>>arrange(-_.n))

In [31]:
ptt_modes.head(10)

,transit_submode,n
0,bus,2999
1,"bus, bus",1706
21,"bus, light_rail",316
2,"bus, bus, bus",302
41,"light_rail, bus, bus",127
40,"light_rail, bus",76
55,"subway, bus",66
32,"bus, subway",63
14,"bus, bus, light_rail",49
33,"bus, subway, bus",46


In [32]:
##### unnesting the transit submode and agencies to get counts. 

In [33]:
ptt_agency_count.head()

,primary_mode,transit_submode,transit_agency,n
0,public_transit,bus,AC TRANSIT,2788
12,public_transit,"bus, bus","San Francisco Municipal Transportation Agency, AC TRANSIT",632
8,public_transit,"bus, bus","AC TRANSIT, San Francisco Municipal Transportation Agency",516
6,public_transit,"bus, bus","AC TRANSIT, AC TRANSIT",263
75,public_transit,"bus, light_rail","AC TRANSIT, San Francisco Municipal Transportation Agency",253


In [34]:
ptt_agency_count['transit_submode'] = ptt_agency_count['transit_submode'].astype(str)
ptt_agency_count['transit_agency'] = ptt_agency_count['transit_agency'].astype(str)

In [35]:
ptt_agency_count['n_modes_taken'] = ptt_agency_count.transit_submode.apply(lambda x: len(x.split()))

In [36]:
ptt_agency_count>>arrange(-_.n_modes_taken)

,primary_mode,transit_submode,transit_agency,n,n_modes_taken
57,public_transit,"bus, bus, bus, rail, bus, bus","San Joaquin Regional Transit District (RTD), San Joaquin Regional Transit District (RTD), San Jo...",1,6
48,public_transit,"bus, bus, bus, bus, bus","San Francisco Municipal Transportation Agency, AC TRANSIT, SolTrans, SolTrans, Sacramento Region...",1,5
49,public_transit,"bus, bus, bus, bus, bus","VTA, SamTrans, AC TRANSIT, SolTrans, Fairfield and Suisun Transit",1,5
50,public_transit,"bus, bus, bus, bus, light_rail","AC TRANSIT, SolTrans, SolTrans, Sacramento Regional Transit, Sacramento Regional Transit",1,5
51,public_transit,"bus, bus, bus, bus, rail","San Francisco Municipal Transportation Agency, San Francisco Municipal Transportation Agency, Sa...",1,5
...,...,...,...,...,...
1,public_transit,bus,Marin Transit,4,1
137,public_transit,subway,Bay Area Rapid Transit,4,1
4,public_transit,bus,SolTrans,3,1
2,public_transit,bus,SamTrans,1,1


In [37]:
## This line of code gets all agencies listed, even if agency is named twice. 
## Need code that counts the unique agencies
#ptt_agency_count['agency_count'] = ptt_agency_count.transit_agency.apply(lambda x: len(x.split(", ")))

In [38]:
ptt_agency_count['agency_count'] = [len(set(x.split(", "))) for x in
                        ptt_agency_count['transit_agency'].str.lower()]

In [39]:
(ptt_agency_count>>arrange(-_.agency_count)).head()

,primary_mode,transit_submode,transit_agency,n,n_modes_taken,agency_count
49,public_transit,"bus, bus, bus, bus, bus","VTA, SamTrans, AC TRANSIT, SolTrans, Fairfield and Suisun Transit",1,5,5
48,public_transit,"bus, bus, bus, bus, bus","San Francisco Municipal Transportation Agency, AC TRANSIT, SolTrans, SolTrans, Sacramento Region...",1,5,4
56,public_transit,"bus, bus, bus, rail, bus","AC TRANSIT, San Francisco Municipal Transportation Agency, San Francisco Municipal Transportatio...",1,5,4
90,public_transit,"bus, rail, bus, bus, bus","The S, Amtrak, AC TRANSIT, AC TRANSIT, San Francisco Municipal Transportation Agency",1,5,4
104,public_transit,"bus, subway, bus, bus","Tri Delta Transit, Bay Area Rapid Transit, San Francisco Municipal Transportation Agency, AC TRA...",1,4,4


In [40]:
ptt_agency_count.n_modes_taken.value_counts()

3    61
4    34
2    28
1    11
5     9
6     1
Name: n_modes_taken, dtype: int64

In [41]:
ptt_agency_count.agency_count.value_counts()

2    66
3    41
1    28
4     8
5     1
Name: agency_count, dtype: int64

In [42]:
ptt_agency_count>>group_by(_.agency_count)>>count(_.n_modes_taken)>>arrange(-_.n)

,agency_count,n_modes_taken,n
5,2,3,29
8,3,3,25
4,2,2,23
9,3,4,13
6,2,4,12
0,1,1,11
2,1,3,7
1,1,2,5
3,1,4,5
12,4,4,4


In [43]:
##### Getting columns for each agency and counts 

In [46]:
ptt_agency_count['transit_agency'].unique().tolist()

['AC TRANSIT',
 'San Francisco Municipal Transportation Agency, AC TRANSIT',
 'AC TRANSIT, San Francisco Municipal Transportation Agency',
 'AC TRANSIT, AC TRANSIT',
 'San Francisco Municipal Transportation Agency, San Francisco Municipal Transportation Agency',
 'San Francisco Municipal Transportation Agency',
 'San Francisco Municipal Transportation Agency, San Francisco Municipal Transportation Agency, AC TRANSIT',
 'San Francisco Municipal Transportation Agency, San Francisco Municipal Transportation Agency, San Francisco Municipal Transportation Agency',
 'Bay Area Rapid Transit, San Francisco Municipal Transportation Agency',
 'San Francisco Municipal Transportation Agency, Bay Area Rapid Transit',
 'AC TRANSIT, San Francisco Municipal Transportation Agency, San Francisco Municipal Transportation Agency',
 'AC TRANSIT, AC TRANSIT, San Francisco Municipal Transportation Agency',
 'San Francisco Municipal Transportation Agency, AC TRANSIT, AC TRANSIT',
 'San Francisco Municipal Tra

In [125]:
## Get just one columns
column = ptt_agency_count[["transit_agency"]]
#remove single-dimensional entries from the shape of an array
col_text = column.squeeze()
# get list of words
text_list = col_text.tolist()
# #join list of words 
text_list = ', '.join(text_list).title()

In [126]:
text_list = text_list.replace(", ", "', '")

In [127]:
text_list = "'" + text_list + "'"

In [128]:
text_list

"'Ac Transit', 'San Francisco Municipal Transportation Agency', 'Ac Transit', 'Ac Transit', 'San Francisco Municipal Transportation Agency', 'Ac Transit', 'Ac Transit', 'Ac Transit', 'San Francisco Municipal Transportation Agency', 'San Francisco Municipal Transportation Agency', 'San Francisco Municipal Transportation Agency', 'San Francisco Municipal Transportation Agency', 'San Francisco Municipal Transportation Agency', 'San Francisco Municipal Transportation Agency', 'Ac Transit', 'San Francisco Municipal Transportation Agency', 'San Francisco Municipal Transportation Agency', 'Ac Transit', 'San Francisco Municipal Transportation Agency', 'San Francisco Municipal Transportation Agency', 'San Francisco Municipal Transportation Agency', 'San Francisco Municipal Transportation Agency', 'San Francisco Municipal Transportation Agency', 'Bay Area Rapid Transit', 'San Francisco Municipal Transportation Agency', 'San Francisco Municipal Transportation Agency', 'Bay Area Rapid Transit', 'S

In [130]:
def unique_list(input_list):
    output_list = []
    for word in input_list:
        if word not in input_list:
            output_list = [word]
            return output_list
    print(output_list)

In [131]:
unique_list(text_list)

[]


In [114]:
## https://stackoverflow.com/questions/46250505/count-the-number-of-times-multiple-substrings-appear-in-dataframe-column

In [31]:
display(HTML("<h4><strong>Trips by Resident Type</strong></h4>")) 


In [30]:
trip_by_res_type = (r_trips
 >>group_by(_.primary_mode,_.trip_taker_resident_type)
 >>summarize(number_trips = _.activity_id.nunique())
 >>arrange(_.primary_mode))

In [32]:
trip_by_res_type

,primary_mode,trip_taker_resident_type,number_trips
0,auto_passenger,core,367739
1,auto_passenger,visitor,23552
2,commercial,NaN,11897
3,on_demand_auto,core,24372
4,on_demand_auto,visitor,2581
5,private_auto,core,232486
6,private_auto,visitor,6204
7,public_transit,core,5986
8,public_transit,visitor,47


In [33]:
chart = (
        alt.Chart(trip_by_res_type)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("number_trips"),
            color=alt.Color("trip_taker_resident_type", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
                                                                         domain=trip_by_res_type["trip_taker_resident_type"].unique().tolist())
            ),
            tooltip=trip_by_res_type.columns.tolist())
      .properties(
    width=800,
    height=500)
        )
chart 

alt.Chart(...)

In [34]:
display(HTML("<h4><strong>Unique Household Ids</strong></h4>")) 

In [35]:
r_trips>>count(_.trip_taker_household_id)>>arrange(-_.n)

,trip_taker_household_id,n
248686,,27897
262322,NaN,16384
110604,8194219563640751815,28
82299,6097771312917788596,27
74914,5552467881171814730,22
...,...,...
262312,9988489366890220188,1
262316,9991342309439330560,1
262317,9992490903429866665,1
262318,9992599661009943006,1


In [36]:
display(HTML("<h4><strong>Checking one household id</strong></h4>")) 


In [37]:
print(len(r_trips>>filter(_.trip_taker_household_id == 8194219563640751815)))
(r_trips>>filter(_.trip_taker_household_id == 8194219563640751815)).sample()

28


,origin_building_use,trip_duration_minutes,trip_taker_language,origin_trct_2020,trip_taker_household_id,trip_taker_sex,trip_taker_home_bgrp_2020,origin_st_2020,trip_taker_home_st_2020,trip_taker_race_ethnicity,transit_agency,transit_route,trip_taker_resident_type,vehicle_type,trip_taker_age,activity_id,destination_bgrp_2020,destination_building_use,trip_taker_wfh,destination_st_2020,destination_cty_2020,trip_taker_work_bgrp_2020,destination_trct_2020,trip_taker_household_income,origin_bgrp_2020,trip_taker_commute_mode,trip_taker_available_vehicles,primary_mode,previous_trip_purpose,trip_taker_building_type,transit_submode,trip_taker_work_st_2020,trip_taker_household_size,origin_cty_2020,destination_land_use,trip_taker_industry,trip_taker_tenure,vehicle_fuel_type,trip_taker_home_trct_2020,trip_taker_work_trct_2020,trip_taker_education,trip_start_time,trip_taker_individual_income,trip_taker_employment_status,trip_purpose,origin_land_use,trip_taker_work_cty_2020,trip_distance_miles,trip_taker_school_grade_attending,trip_taker_home_cty_2020,trip_taker_person_id,trip_end_time
594862,single_family,14,indo_european,"179.03 (San Francisco, CA)",8194219563640751815,female,"1 (Tract 179.03, San Francisco, CA)",California,California,asian_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,22.0,7017187216760344743,"1 (Tract 4028.01, Alameda, CA)",retail,remote,California,Alameda,"1 (Tract 179.03, San Francisco, CA)","4028.01 (Alameda, CA)",1200990.0,"1 (Tract 179.03, San Francisco, CA)",worked_from_home,three_plus,auto_passenger,work_from_home,single_family,NaN,California,10.0,San Francisco,retail,naics62,renter,unknown_fuel_type,"179.03 (San Francisco, CA)","179.03 (San Francisco, CA)",bachelors_degree,16:09:00,76878.0,employed,shop,single_family,San Francisco,7.6,not_attending_school,San Francisco,1325181746552868554,16:23:22


In [38]:
## checking one household id
r_trips>>filter(_.trip_taker_household_id == 8194219563640751815)>>count(_.trip_taker_person_id, _.trip_taker_age,
                                                                         _.trip_taker_sex, _.trip_taker_household_size)

,trip_taker_person_id,trip_taker_age,trip_taker_sex,trip_taker_household_size,n
0,1325181746552868554,22.0,female,10.0,6
1,2310331066900902679,30.0,female,10.0,6
2,4746831412975349070,34.0,female,10.0,2
3,10066255028734967962,28.0,male,10.0,4
4,11058477761487230854,32.0,male,10.0,2
5,12894946300265742193,39.0,male,10.0,2
6,14409692340574959811,27.0,male,10.0,6


#### Adding in Replica's People Dataset

In [39]:
r_ppl = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_ppl}"))

/tmp/ipykernel_320/1951040339.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  r_ppl = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_ppl}"))


replica people dataframe sample

In [40]:
r_ppl.sample(5)

,person_id,work_st_2020,household_income,resident_type,language,available_vehicles,race_ethnicity,education,work_bgrp_2020,wfh,home_cty_2020,home_trct_2020,tenure,industry,work_trct_2020,school_grade_attending,building_type,commute_mode,employment_status,work_cty_2020,individual_income,home_st_2020,sex,household_size,home_bgrp_2020,household_id,age
126764,13176772315533876677,California,275131.0,core,asian_pacific,two,asian_not_hispanic_or_latino,bachelors_degree,"3 (Tract 117, San Francisco, CA)",in_person,Alameda,"4415.21 (Alameda, CA)",owner,naics54,"117 (San Francisco, CA)",not_attending_school,single_family,public_transit,employed,San Francisco,207687.0,California,male,3.0,"3 (Tract 4415.21, Alameda, CA)",4405505003628472113,61.0
231078,5369653974752218591,California,67771.0,core,spanish,one,hispanic_or_latino_origin,k_12,"3 (Tract 254.01, San Francisco, CA)",in_person,Contra Costa,"3072.02 (Contra Costa, CA)",renter,naics23,"254.01 (San Francisco, CA)",not_attending_school,multiple_units,private_auto,employed,San Francisco,37165.0,California,male,2.0,"3 (Tract 3072.02, Contra Costa, CA)",2520287356701498339,35.0
45631,12611646703074941567,California,223094.0,core,english,two,two_races_not_hispanic_or_latino,some_college,"2 (Tract 6023, San Mateo, CA)",in_person,Alameda,"4213 (Alameda, CA)",renter,naics5412,"6023 (San Mateo, CA)",not_attending_school,single_family,private_auto,employed,San Mateo,153083.0,California,male,3.0,"3 (Tract 4213, Alameda, CA)",17976817071532499180,35.0
106775,8691085808549329285,California,22658.0,core,english,one,white_not_hispanic_or_latino,advanced_degree,"3 (Tract 607.01, San Francisco, CA)",in_person,Alameda,"4037.01 (Alameda, CA)",renter,naics54192,"607.01 (San Francisco, CA)",graduate,multiple_units,private_auto,employed,San Francisco,22658.0,California,female,1.0,"2 (Tract 4037.01, Alameda, CA)",8374180285393209524,25.0
251509,16288548793878627261,California,614473.0,core,spanish,three_plus,white_not_hispanic_or_latino,advanced_degree,"2 (Tract 105, San Francisco, CA)",in_person,Contra Costa,"3462.06 (Contra Costa, CA)",owner,naics52,"105 (San Francisco, CA)",not_attending_school,single_family,public_transit,employed,San Francisco,614473.0,California,male,3.0,"1 (Tract 3462.06, Contra Costa, CA)",16466378101748887324,54.0


In [41]:
r_ppl>>group_by(_.resident_type)>>summarize(_unique_ids = _.person_id.nunique())

,resident_type,_unique_ids
0,core,278158
1,visitor,23034


In [42]:
(r_ppl
 >>filter(_.home_bgrp_2020!="Visitor (no home location)")
 >>group_by(_.commute_mode, _.sex)
 >>summarize(
     n_ppl = _.person_id.nunique(),
     avg_h_income = _.household_income.mean(),
     avg_p_income = _.individual_income.mean(),
     avg_age = _.age.mean())
)

,commute_mode,sex,n_ppl,avg_h_income,avg_p_income,avg_age
0,auto_passenger,female,10908,175717.982765,70255.626054,42.234507
1,auto_passenger,male,11605,177241.394916,82072.291685,42.185523
2,biking,female,234,248565.141026,82592.418803,36.085470
3,biking,male,527,219563.083491,120152.859583,38.426945
4,other_travel_mode,female,36176,126129.721003,21413.682939,55.161599
5,other_travel_mode,male,26290,116775.549106,33563.078813,54.328262
6,private_auto,female,45312,186499.659406,76492.005341,42.626302
7,private_auto,male,59762,190643.201265,100712.386918,42.872093
8,public_transit,female,22008,193540.921347,84074.072110,41.921937
9,public_transit,male,26480,207838.001699,125445.942485,42.259290


In [43]:
(r_trips
 >>filter(_.trip_taker_home_bgrp_2020!="Visitor (no home location)")
 >>group_by(_.primary_mode, _.trip_taker_sex)
 >>summarize(
     n_ppl = _.trip_taker_person_id.nunique(),
     avg_h_income = _.trip_taker_household_income.mean(),
     avg_p_income = _.trip_taker_individual_income.mean(),
     avg_age = _.trip_taker_age.mean())
)
     

,primary_mode,trip_taker_sex,n_ppl,avg_h_income,avg_p_income,avg_age
0,auto_passenger,female,99350,173666.662228,65104.398490,45.064633
1,auto_passenger,male,105359,187590.985442,97296.509379,43.991868
2,commercial,NaN,1,NaN,NaN,NaN
3,on_demand_auto,female,9051,186226.833189,79990.395102,42.900399
4,on_demand_auto,male,10159,202467.780725,111915.392968,42.436605
5,private_auto,female,68268,174867.937251,62299.064752,45.316785
6,private_auto,male,75223,182301.952228,90322.373466,44.265320
7,public_transit,female,2256,186720.499643,82496.554922,39.104494
8,public_transit,male,2535,205334.441232,114041.914833,38.312382


### Read in Streetlight Data: 2022

In [44]:
streetlight = "streetlight_bay_bridge_corridor_study_corridor_study.csv"

In [45]:
sl_data = to_snakecase(pd.read_csv(f"{GCS_PATH}{streetlight}"))

In [46]:
sl_data.sample(5)

,data_periods,mode_of_travel,zone_id,zone_name,road_classification,line_zone_length__miles_,zone_is_pass_through,zone_direction__degrees_,zone_cardinal_direction,zone_is_bi_direction,day_type,day_part,average_daily_segment_traffic__stl_volume_,avg_segment_speed__mph_,avg_segment_travel_time__sec_,free_flow_speed__mph_,vehicle_miles_of_travel__stl_volume_,travel_time_index,congested_segment,_85th_speed_percentile
1655,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1135393455,I 80 / 781148922 / 1,Motorway,0.256,yes,265,WEST,no,1: Weekday (M-F),05: 3am (3am-4am),1285,67,13,67.716,329.06,1.01,False,74
1064,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1118036302,I 80 / 839972875 / 1,Motorway,0.006,yes,265,WEST,no,2: Weekend Day (Sa-Su),14: Mid-Day (10am-4pm),38719,50,2,68.762,214.99,1.38,True,62
441,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1036056766,San Francisco – Oakland Bay Bridge / 52721870 / 1,Motorway,0.073,yes,37,EAST,no,2: Weekend Day (Sa-Su),21: Peak PM (4pm-8pm),37569,44,6,67.267,2733.58,1.53,True,60
215,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1022959400,San Francisco – Oakland Bay Bridge / 661905446...,Motorway,1.939,yes,40,EAST,no,1: Weekday (M-F),05: 3am (3am-4am),577,63,108,64.627,1118.62,1.02,False,71
902,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1117259749,San Francisco – Oakland Bay Bridge / 237731428...,Motorway,1.961,yes,80,EAST,no,0: All Days (M-Su),02: Early AM (12am-6am),9063,69,102,70.594,17770.60,1.02,False,77


#### Data Explorations: Streetlight

In [47]:
len(sl_data)

1710

In [48]:
sl_data>>count(_.zone_id, _.zone_name)

,zone_id,zone_name,n
0,1005898592,San Francisco – Oakland Bay Bridge / 8921938 / 1,90
1,1020456908,San Francisco – Oakland Bay Bridge / 23874736 / 1,90
2,1022959400,San Francisco – Oakland Bay Bridge / 661905446...,90
3,1032819756,San Francisco – Oakland Bay Bridge / 11415208 / 1,90
4,1036056766,San Francisco – Oakland Bay Bridge / 52721870 / 1,90
5,1050419851,San Francisco – Oakland Bay Bridge / 179235221...,90
6,1093181568,San Francisco – Oakland Bay Bridge / 50691047 / 1,90
7,1112052563,San Francisco – Oakland Bay Bridge / 236348360...,90
8,1112062603,San Francisco – Oakland Bay Bridge / 236348361...,90
9,1114495634,I 80 / 395419317 / 1,90


In [49]:
sl_data>>filter(_.zone_id==1133975975)>>count(_.day_part)

,day_part,n
0,00: All Day (12am-12am),3
1,01: 12am (12am-1am),3
2,02: Early AM (12am-6am),3
3,03: 1am (1am-2am),3
4,04: 2am (2am-3am),3
5,05: 3am (3am-4am),3
6,06: 4am (4am-5am),3
7,07: 5am (5am-6am),3
8,08: 6am (6am-7am),3
9,09: Peak AM (6am-10am),3


* The data here shows that each segment is broken out into hour time slots and then also aggregated to peak time periods. moving forward we will look at the `All Day` for `day_part` and `All Days` for `day_type`

In [50]:
sl_data_single_day = sl_data>>filter(_.day_part == '00: All Day (12am-12am)', _.day_type == '0: All Days (M-Su)')

In [51]:
len(sl_data_single_day)

19

In [52]:
sl_data_single_day>>count(_.zone_id, _.zone_name)

,zone_id,zone_name,n
0,1005898592,San Francisco – Oakland Bay Bridge / 8921938 / 1,1
1,1020456908,San Francisco – Oakland Bay Bridge / 23874736 / 1,1
2,1022959400,San Francisco – Oakland Bay Bridge / 661905446...,1
3,1032819756,San Francisco – Oakland Bay Bridge / 11415208 / 1,1
4,1036056766,San Francisco – Oakland Bay Bridge / 52721870 / 1,1
5,1050419851,San Francisco – Oakland Bay Bridge / 179235221...,1
6,1093181568,San Francisco – Oakland Bay Bridge / 50691047 / 1,1
7,1112052563,San Francisco – Oakland Bay Bridge / 236348360...,1
8,1112062603,San Francisco – Oakland Bay Bridge / 236348361...,1
9,1114495634,I 80 / 395419317 / 1,1


In [53]:
sl_data_single_day>>filter(_.zone_id==1133975975)

,data_periods,mode_of_travel,zone_id,zone_name,road_classification,line_zone_length__miles_,zone_is_pass_through,zone_direction__degrees_,zone_cardinal_direction,zone_is_bi_direction,day_type,day_part,average_daily_segment_traffic__stl_volume_,avg_segment_speed__mph_,avg_segment_travel_time__sec_,free_flow_speed__mph_,vehicle_miles_of_travel__stl_volume_,travel_time_index,congested_segment,_85th_speed_percentile
1530,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1133975975,San Francisco – Oakland Bay Bridge / 929579737...,Motorway,0.273,yes,215,WEST,no,0: All Days (M-Su),00: All Day (12am-12am),128139,49,23,68.634,35038.3,1.39,True,67


In [54]:
sl_data_single_day.sample(3)

,data_periods,mode_of_travel,zone_id,zone_name,road_classification,line_zone_length__miles_,zone_is_pass_through,zone_direction__degrees_,zone_cardinal_direction,zone_is_bi_direction,day_type,day_part,average_daily_segment_traffic__stl_volume_,avg_segment_speed__mph_,avg_segment_travel_time__sec_,free_flow_speed__mph_,vehicle_miles_of_travel__stl_volume_,travel_time_index,congested_segment,_85th_speed_percentile
1620,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1135393455,I 80 / 781148922 / 1,Motorway,0.256,yes,265,WEST,no,0: All Days (M-Su),00: All Day (12am-12am),104573,53,18,67.716,26778.50,1.28,True,67
1260,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1128174334,San Francisco – Oakland Bay Bridge / 237731428...,Motorway,0.177,yes,43,EAST,no,0: All Days (M-Su),00: All Day (12am-12am),143645,55,12,67.749,25402.40,1.23,False,66
1080,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1120995048,San Francisco – Oakland Bay Bridge / 497579295...,Motorway,0.056,yes,222,WEST,no,0: All Days (M-Su),00: All Day (12am-12am),127886,48,4,68.206,7121.77,1.41,True,67


In [55]:
sl_data_single_day>>group_by(_.zone_cardinal_direction, _.day_type, _.day_part)>>summarize(avg_speed= _.avg_segment_speed__mph_.mean(),
                                                                                          avg_travel_time = _.avg_segment_travel_time__sec_.mean(),
                                                                                          avg_volume = _.average_daily_segment_traffic__stl_volume_.mean(),
                                                                                          sum_volume = _.average_daily_segment_traffic__stl_volume_.sum())

,zone_cardinal_direction,day_type,day_part,avg_speed,avg_travel_time,avg_volume,sum_volume
0,EAST,0: All Days (M-Su),00: All Day (12am-12am),55.900000,32.600000,139682.300000,1396823
1,WEST,0: All Days (M-Su),00: All Day (12am-12am),50.333333,41.777778,123029.777778,1107268


In [56]:
sl_peaks = (sl_data>>filter(
                            _.day_part != "01: 12am (12am-1am)",
                            _.day_part != "03: 1am (1am-2am)",
                            _.day_part != "04: 2am (2am-3am)",
                            _.day_part != "05: 3am (3am-4am)",
                            _.day_part != "06: 4am (4am-5am)",
                            _.day_part != "07: 5am (5am-6am)",
                            _.day_part != "08: 6am (6am-7am)",
                            _.day_part != "10: 7am (7am-8am)",
                            _.day_part != "11: 8am (8am-9am)",
                            _.day_part != "12: 9am (9am-10am)",
                            _.day_part != "13: 10am (10am-11am)",
                            _.day_part != "15: 11am (11am-12noon)",
                            _.day_part != "16: 12pm (12noon-1pm)",
                            _.day_part != "17: 1pm (1pm-2pm)",
                            _.day_part != "18: 2pm (2pm-3pm)",
                            _.day_part != "19: 3pm (3pm-4pm)",
                            _.day_part != "20: 4pm (4pm-5pm)",
                            _.day_part != "22: 5pm (5pm-6pm)",
                            _.day_part != "23: 6pm (6pm-7pm)",
                            _.day_part != "24: 7pm (7pm-8pm)",
                            _.day_part != "25: 8pm (8pm-9pm)",
                            _.day_part != "27: 9pm (9pm-10pm)",
                            _.day_part != "28: 10pm (10pm-11pm)",
                            _.day_part !="29: 11pm (11pm-12am)"))

In [57]:
sl_hourly = (sl_data>>filter(_.day_part != "00: All Day (12am-12am)",
                            _.day_part != "02: Early AM (12am-6am)",
                            _.day_part != "09: Peak AM (6am-10am)",
                            _.day_part != "14: Mid-Day (10am-4pm)",
                            _.day_part != "21: Peak PM (4pm-8pm)", _.day_part !="26: Late PM (8pm-12am)"))

In [58]:
sl_peaks_agg = sl_peaks>>group_by(_.zone_cardinal_direction,
    _.day_type, _.day_part)>>summarize(avg_speed= _.avg_segment_speed__mph_.mean(),
                                                                                          avg_travel_time = _.avg_segment_travel_time__sec_.mean(),
                                                                                          avg_volume = _.average_daily_segment_traffic__stl_volume_.mean(),
                                                                                          sum_volume = _.average_daily_segment_traffic__stl_volume_.sum())

In [59]:
sl_peaks_agg.sample()

,zone_cardinal_direction,day_type,day_part,avg_speed,avg_travel_time,avg_volume,sum_volume
34,WEST,2: Weekend Day (Sa-Su),21: Peak PM (4pm-8pm),40.222222,54.222222,23846.222222,214616


In [60]:
chart = (
        alt.Chart(sl_peaks_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("avg_speed"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_peaks_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_peaks_agg.columns.tolist(),
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [61]:
chart = (
        alt.Chart(sl_peaks_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("avg_speed"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_peaks_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_peaks_agg.columns.tolist(),
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [62]:
sl_hourly_agg = sl_hourly>>filter(_.day_type=="0: All Days (M-Su)")>>group_by(_.zone_cardinal_direction,
    _.day_type, _.day_part)>>summarize(avg_speed= _.avg_segment_speed__mph_.mean(),
                                                                                          avg_travel_time = _.avg_segment_travel_time__sec_.mean(),
                                                                                          avg_volume = _.average_daily_segment_traffic__stl_volume_.mean(),
                                                                                          sum_volume = _.average_daily_segment_traffic__stl_volume_.sum())

In [63]:
sl_hourly_agg.sample()

,zone_cardinal_direction,day_type,day_part,avg_speed,avg_travel_time,avg_volume,sum_volume
46,WEST,0: All Days (M-Su),28: 10pm (10pm-11pm),63.333333,30.666667,3115.0,28035


In [64]:
chart = (
        alt.Chart(sl_hourly_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("avg_speed"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_hourly_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_hourly_agg.columns.tolist(),
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [65]:
chart = (
        alt.Chart(sl_hourly_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("sum_volume"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_hourly_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_hourly_agg.columns.tolist(),
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)